In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from utils.towncentre import load_towncentre, aug_data
from utils.angles import deg2bit, bit2deg, bit2deg_multi
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True, verbose=1)

image_height, image_width = xtr.shape[1], xtr.shape[2]

ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)
yte_rad = np.deg2rad(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

Number of train samples: 6916
Number of validation samples: 874
Number of test samples: 904


#### We will model our distribution via

$ p(\phi | x) =  \sum_{i=1}^{k}{p(z_i|x) p(\phi|z_i,x)}$

In [28]:
from keras.layers.merge import concatenate
from keras import backend as K

In [55]:
from models.vgg_vmmix import BiternionVGGMixture
vgg_vmmix = BiternionVGGMixture()

In [37]:
y_true = yte_bit
y_preds = vgg_vmmix.model.predict(xte)

In [174]:
y_true = yte_bit
y_preds = vgg_vmmix.model.predict(xte)

y_true_tf = tf.placeholder(tf.float32, shape=[None, 2])
y_preds_tf = tf.placeholder(tf.float32, shape=[None, 5, 4])

loglike = vgg_vmmix._von_mises_mixture_log_likelihood_tf(y_true_tf, y_preds_tf)

In [176]:
with tf.Session() as sess:
    loglike_values = loglike.eval(feed_dict={y_true_tf: y_true, y_preds_tf: y_preds})

In [177]:
loglike_values

array([-1.85568798, -1.84363604, -1.84949982, -1.81935668, -1.82011247,
       -1.82928598, -1.83200419, -1.83633208, -1.85226727, -1.85718977,
       -1.81339574, -1.81241727, -1.81984401, -1.81136048, -1.81087577,
       -1.86146784, -1.85886967, -1.86218166, -1.86197627, -1.83321989,
       -1.83756244, -1.83503616, -1.83740652, -1.83446121, -1.83203959,
       -1.83204031, -1.83282411, -1.83190429, -1.85035396, -1.82843709,
       -1.83132505, -1.84075809, -1.85198534, -1.84987831, -1.84478366,
       -1.84712183, -1.85710394, -1.85685372, -1.849123  , -1.8468492 ,
       -1.84798145, -1.84646797, -1.84526742, -1.82918286, -1.81961858,
       -1.81842375, -1.84034717, -1.83427525, -1.83334494, -1.82622027,
       -1.82529485, -1.8406353 , -1.8333571 , -1.82535994, -1.8566134 ,
       -1.86631739, -1.84538424, -1.83393478, -1.83307433, -1.83317471,
       -1.83306789, -1.85641921, -1.85943377, -1.85677481, -1.85952306,
       -1.85949039, -1.8281194 , -1.83045816, -1.82541871, -1.82